In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
## RFM => Regency (Último -> data - data - 1) / Frequency (quantidade -> n) / Monetary (soma/média -> R   ~x)

def rfm_variables(df):
    ## FREQUENCY
    f_sales = len(df) ## Quantidade de vendas 
    f_profit = len(df[df['Profit'] > 0]) ## Quantidade de vendas com lucros
    ## MONETARY
    m_sales = round(df['Sales'].sum(), 2) ## Total de vendas
    m_profit = round(df['Profit'].sum(), 2) ## Total de vendas com lucro
    m_quantity = df['Quantity'].sum() ## Total de quantidades
    m_deliver = round(df['Shipping Cost'].sum(), 2) ## Total do custo de entregas
    ## PERIODICITY
    df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates() ## Periodicidade
    df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
    df_sort['diff_int'] = df_sort['diff'].dt.days
    ## REGENCY
    r_days = round(df_sort['diff_int'].mean(), 2)
    return f_sales, f_profit, m_sales, m_profit, m_quantity, m_deliver, r_days

In [3]:
def fit_data(data, variable):
    rfm = pd.DataFrame()
    variables = data[variable].unique()
    for v in variables:
        var = data[data[variable] == v]
        f_sales, f_profit, m_sales, m_profit, m_quantity, m_deliver, r_days = rfm_variables(var)
        rfm = rfm.append(
            {
                'reference': v,
                'm_sales': m_sales,
                'm_profit': m_profit, 
                'm_quantity': m_quantity,
                'm_deliver': m_deliver,
                'r_days': r_days,
                'f_sales': f_sales, 
                'f_profit': f_profit
            },
            ignore_index = True
        )
    return rfm

In [4]:
data = pd.read_feather('database/GS.feather')
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,...,OMonth,OPeriod,Order Date Month,SYear,SMonth,SPeriod,Ship Date Month,Delivery,Price,Benefit
0,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,Mexico,...,10,46,2014-10-01,2014,10,46,2014-10-01,4,4.36,1
1,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,31.52,1
2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,96.64,1
3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,8.86,1
4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,35.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,11.07,1
51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,30.72,1
51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,20.13,1
51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,32.61,1


# AULA 09 - Clusterização

In [5]:
variables = ['m_sales', 'm_profit', 'm_quantity', 'm_deliver', 'r_days', 'f_sales', 'f_profit']

In [6]:
country_rfm = fit_data(data, 'Country')
country_rfm = country_rfm.fillna(0)
country_rfm['cluster'] = KMeans(
    n_clusters=3,
    random_state=0
).fit(
    country_rfm[
        variables
    ]
).labels_

country_rfm

/tmp/ipykernel_4532/2535250959.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_4532/2535250959.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_4532/2535250959.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_4532/2535250959.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_4532/2535250959.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_4532/2535250959.py:7: FutureWarning

,reference,m_sales,m_profit,m_quantity,m_deliver,r_days,f_sales,f_profit,cluster
0,Mexico,622590.62,102818.10,10011,67658.40,1.89,2644,2272,2
1,Colombia,81502.53,18798.05,1277,9411.77,9.32,360,350,0
2,Brazil,361106.42,30090.50,6148,38170.04,2.66,1599,1185,2
3,Nicaragua,149687.06,33401.44,2235,17155.17,5.30,614,600,0
4,Panama,51539.93,-17723.45,1426,5416.04,7.78,388,75,0
...,...,...,...,...,...,...,...,...,...
142,Ethiopia,850.62,290.16,18,81.54,358.67,7,7,0
143,Equatorial Guinea,150.51,44.46,3,9.38,0.00,3,3,0
144,Bahrain,669.18,12.84,6,72.76,0.00,2,2,0
145,Chad,1317.03,90.00,11,297.94,0.00,2,1,0


In [7]:
cluster = pd.DataFrame()

for index, row in enumerate(KMeans(n_clusters=3, random_state=0).fit(country_rfm[variables]).cluster_centers_):
    cluster = cluster.append(
        {
            'cluster': index,
            'clm_sales': row[0],
            'clm_profit': row[1],
            'clm_quantity': row[2],
            'clm_deliver': row[3],
            'clr_days': row[4],
            'clf_sales': row[5],
            'clf_profit': row[6]
        },
        ignore_index= True 
    ) 

cluster

/tmp/ipykernel_4532/1361659738.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
/tmp/ipykernel_4532/1361659738.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
/tmp/ipykernel_4532/1361659738.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(


,cluster,clm_sales,clm_profit,clm_quantity,clm_deliver,clr_days,clf_sales,clf_profit
0,0.0,3.448848e+04,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
1,1.0,2.297201e+06,286397.020000,37873.000000,238173.790000,1.180000,9994.000000,8058.000000
2,2.0,6.244867e+05,95603.512222,7735.333333,67089.675556,2.285556,2047.777778,1645.333333


In [8]:
country_rfm = country_rfm.merge(cluster, on='cluster', how='left')
country_rfm

,reference,m_sales,m_profit,m_quantity,m_deliver,r_days,f_sales,f_profit,cluster,clm_sales,clm_profit,clm_quantity,clm_deliver,clr_days,clf_sales,clf_profit
0,Mexico,622590.62,102818.10,10011,67658.40,1.89,2644,2272,2,624486.657778,95603.512222,7735.333333,67089.675556,2.285556,2047.777778,1645.333333
1,Colombia,81502.53,18798.05,1277,9411.77,9.32,360,350,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
2,Brazil,361106.42,30090.50,6148,38170.04,2.66,1599,1185,2,624486.657778,95603.512222,7735.333333,67089.675556,2.285556,2047.777778,1645.333333
3,Nicaragua,149687.06,33401.44,2235,17155.17,5.30,614,600,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
4,Panama,51539.93,-17723.45,1426,5416.04,7.78,388,75,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Ethiopia,850.62,290.16,18,81.54,358.67,7,7,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
143,Equatorial Guinea,150.51,44.46,3,9.38,0.00,3,3,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
144,Bahrain,669.18,12.84,6,72.76,0.00,2,2,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496
145,Chad,1317.03,90.00,11,297.94,0.00,2,1,0,34488.475109,2340.355036,516.941606,3728.721314,111.390073,166.905109,111.036496


In [14]:
country_rfm.to_feather('database/clusterizacao_pais.feather')